---
# Deep Learning

## Project: Speed-prediction based on video input

---
## Step 0: Import required packages

In [1]:
## LIST OF ALL IMPORTS
import os
import sys
import csv
import math
import random
import time
import os.path as path
from datetime import datetime
import skvideo.io
import pickle, h5py
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import cv2
import tensorflow as tf

import imgaug as ia # Image augmentation for machine learning
from imgaug import augmenters as iaa

import keras
from keras import backend as K
import tensorflow as tf

from keras.models import Model,Sequential
from keras.layers.core import Flatten,Dropout,Activation,Dense
from keras.layers.convolutional import Conv3D, MaxPooling3D, ZeroPadding3D
from keras.layers.normalization import BatchNormalization
from keras.optimizers import SGD
from keras.callbacks import TensorBoard, EarlyStopping, ModelCheckpoint

from utils import create_image, display, Data_BatchGenerator, load_weights, squared_loss 
# Visualizations will be shown in the notebook.
%matplotlib inline 

Using TensorFlow backend.


---
## Step 1: Create Dataset

In [2]:
repeat=0
train_video_address='dataset/data/train.mp4'
video_label_address='dataset/data/train.txt'
train_images_address='dataset/video_frames/images/'
train_labels_address='dataset/video_frames/train_labels.txt'

if repeat==1:## Create image frames
    create_image(train_video_address,video_label_address,train_images_address,train_labels_address)
else:
    print("Skipping.")

Skipping.


In [3]:
print('Using train images at {}, train labels at {}.'.format(train_images_address,train_labels_address))
# train_images_address='dataset/video_frames/images/'
# train_labels_address='dataset/video_frames/train_labels.txt'
H,W= 112,112
timestep=20
root_path='dataset/video_frames/'
train_images=os.listdir(train_images_address)
time_indices=list(range(timestep-1,len(train_images),timestep))
batch_size=5

Using train images at dataset/video_frames/images/, train labels at dataset/video_frames/train_labels.txt.


----

## Step 2: Design and Test a Model Architecture

Two model choices- C3D networks (Learning Spatiotemporal Features with 3D Convolutional Networks) or a CNN-RNN combination to analyze semantic scenes, and then process sequential data.

In [4]:
input_shape=(timestep,H,W,3)
keras.__version__ # 2.0.5 desired

'2.0.5'

In [5]:
model=Sequential()

# Layer One
model.add(Conv3D(64,(3,3,3),input_shape=input_shape,activation='relu',padding='same',name='Conv3D_1'))
model.add(MaxPooling3D(pool_size=(1,2,2),strides=(1,2,2),padding='valid',name='MaxPool3D_1'))
# Layer Two
model.add(Conv3D(128,(3,3,3),activation='relu',padding='same',name='Conv3D_2'))
model.add(MaxPooling3D(pool_size=(2,2,2),strides=(2,2,2),padding='valid',name='MaxPool3D_2'))
# Layer Three
model.add(Conv3D(256,(3,3,3),activation='relu',padding='same',name='Conv3D_3_1'))
model.add(Conv3D(256,(3,3,3),activation='relu',padding='same',name='Conv3D_3_2'))
model.add(MaxPooling3D(pool_size=(2,2,2),strides=(2,2,2),padding='valid',name='MaxPool3D_3'))
# Layer Four
model.add(Conv3D(512,(3,3,3),activation='relu',padding='same',name='Conv3D_4_1'))
model.add(Conv3D(512,(3,3,3),activation='relu',padding='same',name='Conv3D_4_2'))
model.add(MaxPooling3D(pool_size=(2,2,2),strides=(2,2,2),padding='valid',name='MaxPool3D_4'))
# Layer Five
model.add(Conv3D(512,(3,3,3),activation='relu',padding='same',name='Conv3D_5_1'))
model.add(Conv3D(512,(3,3,3),activation='relu',padding='same',name='Conv3D_5_2'))
model.add(ZeroPadding3D(padding=((0,0),(0,1),(0,1)),name='ZeroPadding3D_1'))
model.add(MaxPooling3D(pool_size=(2,2,2),strides=(2,2,2),padding='valid',name='MaxPool3D_5'))
# Layer Six
model.add(Flatten())
model.add(Dense(4096,activation='relu',name='FullConnected_1'))
model.add(Dropout(0.5))
# Layer Seven
model.add(Dense(4096,activation='relu',name='FullConnected_2'))
model.add(Dropout(0.5))
# Layer Eight (Final Layer)
model.add(Dense(1,activation='linear',name='Output_layer'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Conv3D_1 (Conv3D)            (None, 16, 112, 112, 64)  5248      
_________________________________________________________________
MaxPool3D_1 (MaxPooling3D)   (None, 16, 56, 56, 64)    0         
_________________________________________________________________
Conv3D_2 (Conv3D)            (None, 16, 56, 56, 128)   221312    
_________________________________________________________________
MaxPool3D_2 (MaxPooling3D)   (None, 8, 28, 28, 128)    0         
_________________________________________________________________
Conv3D_3_1 (Conv3D)          (None, 8, 28, 28, 256)    884992    
_________________________________________________________________
Conv3D_3_2 (Conv3D)          (None, 8, 28, 28, 256)    1769728   
_________________________________________________________________
MaxPool3D_3 (MaxPooling3D)   (None, 4, 14, 14, 256)    0         
__________

In [6]:
print("Loading weights from Sports-1M dataset.")
pretrained_weights_address='dataset/c3d_pretrainedweights.h5'
trained_weights_address='c3d_weights.h5'
load_weights(model,pretrained_weights_address)
model.load_weights(trained_weights_address)
print("Loaded weights.")

Loading weights from Sports-1M dataset.
Loaded weights.


In [7]:
train_validation_split=0.95
print("Retraining with a train-validation ratio of {}.".format(train_validation_split))
retrained_weights='c3d_weights.h5'

train_indices=list(time_indices[0:int(len(time_indices)*train_validation_split)])
validation_indices=list(time_indices[int(len(time_indices)*train_validation_split):])

early_stopping=EarlyStopping(monitor='val_loss',min_delta=0.00001,patience=5,mode='min',verbose=1)
model_checkpoints=ModelCheckpoint(retrained_weights,monitor='val_loss',save_best_only=True,mode='min',verbose=1,period=1)

train_generator=Data_BatchGenerator(root_path,train_images_address,train_labels_address,indices=train_indices,batch=batch_size,timestep=timestep,augment=True,normalization=True)
validation_generator=Data_BatchGenerator(root_path,train_images_address,train_labels_address,indices=validation_indices,batch=batch_size,timestep=timestep,augment=False,normalization=True)

print("Generators created.")

Retraining with a train-validation ratio of 0.95.
Generators created.


In [8]:
# Creating a graph
# merged=tf.merge_all_summaries/()
# writer=tf.train.SummaryWriter('./graph-logs',sess.graph_def)

# merged=tf.summary.merge_all()
# writer=tf.summary.FileWriter('./graph-logs',sess.graph_def)

In [ ]:
print("Compiling and running model.")
#model.load_weights(retrained_weights)
log='logs/speed/'
learning_rate=0.00001
momentum=0.9
learning_rate_decay=0.0005


# count=max([int(n) for n in os.listdir(log)] or [0]) +1
init_tensorboard=TensorBoard(log_dir='./logs',histogram_freq=0,write_graph=True,write_images=False)
stochastic_gd=SGD(lr=learning_rate,decay=learning_rate_decay,momentum=momentum)
print("Compiling model.")

epochs=50
verbose=1
q_size=10
callbacks=[early_stopping,model_checkpoints,init_tensorboard]

model.compile(loss=squared_loss,optimizer=stochastic_gd)
model.fit_generator(generator=train_generator.generator(),steps_per_epoch=train_generator.sizeof(),
                    epochs=epochs,verbose=verbose,callbacks=callbacks,max_q_size=q_size,
                    validation_data=validation_generator.generator(),validation_steps=validation_generator.sizeof())
print("Completed.")

In [ ]:
# Compiling and running model.
# Compiling model.
# Epoch 1/5
# 216/216 [============================>.] - ETA: 40s - loss: 75.3386Epoch 00000: val_loss improved from inf to 21.04785, saving model to c3d_weights.h5
# 217/216 [==============================] - 11882s - loss: 75.1293 - val_loss: 21.0478
# Epoch 2/5
# 216/216 [============================>.] - ETA: 39s - loss: 24.3069Epoch 00001: val_loss improved from 21.04785 to 13.94606, saving model to c3d_weights.h5
# 217/216 [==============================] - 11790s - loss: 24.2135 - val_loss: 13.9461
# Epoch 3/5
# 216/216 [============================>.] - ETA: 39s - loss: 9.9180Epoch 00002: val_loss improved from 13.94606 to 12.66372, saving model to c3d_weights.h5
# 217/216 [==============================] - 11787s - loss: 9.9208 - val_loss: 12.6637
# Epoch 4/5
# 216/216 [============================>.] - ETA: 39s - loss: 5.9344Epoch 00003: val_loss improved from 12.66372 to 7.66480, saving model to c3d_weights.h5
# 217/216 [==============================] - 11800s - loss: 5.9245 - val_loss: 7.6648
# Epoch 5/5
# 216/216 [============================>.] - ETA: 39s - loss: 3.9967Epoch 00004: val_loss improved from 7.66480 to 6.85855, saving model to c3d_weights.h5
# 217/216 [==============================] - 11796s - loss: 4.0138 - val_loss: 6.8585
# Completed.

---

## Step 3: Evaluation on train/test videos

In [11]:
# train_video_address='dataset/data/train.mp4'
# train_labels_address='dataset/video_frames/train_labels.txt'
output_root='output/'
model_output_video=output_root+'trained_video_output.avi'
model_output_labels=output_root+'predicted_output.txt'

vreader=skvideo.io.vreader(train_video_address)

train_labels=open(train_labels_address,'r').readlines()
write_prediction=open(model_output_labels,'w')
vreader_shape=[480,640,3]
h=vreader_shape[0]
w=vreader_shape[1]
vwriter=cv2.VideoWriter(model_output_video,cv2.VideoWriter_fourcc(*'MPEG'),20.,(w,h))

batch_x_video=np.zeros((timestep,h,w,3))
frame_t=np.zeros((1,timestep,H,W,3))
frame_count=0
count=0
sum_frames=0
predicted_speed=0.
mse_error=0.
while(True):
    for frame in vreader:
        print('Frame {}'.format(count))
        if sum_frames>-1:
            batch_x_video[count]=frame
            height=frame.shape[0]
            frame=frame[int(height/3):int(height),:,:]
            frame=cv2.resize(frame.copy(),(H,W))
            frame=frame-[105,115,120]
            frame_t[0,count]=frame

            if(count==(timestep-1)):
                predicted_speed=model.predict(frame_t)[0][0]
                count=-1
                
                for ind in range(timestep):
                    frame=batch_x_video[ind]

                    write_speed_predicted='Predicted speed: '+str(round(predicted_speed,2))
                    frame=cv2.putText(frame,write_speed_predicted,(10,20), cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,0,255),2)
                                        
                    #Assuming we are given actual speeds
                    true_speed=float(train_labels[10*sum_frames+ind].strip())
                    
                    write_speed_actual='Actual speed: '+str(round(true_speed,2))
                    frame=cv2.putText(frame,write_speed_actual,(10,50), cv2.FONT_HERSHEY_SIMPLEX,0.8,(0,0,255),2)

                    mse_error=((true_speed-predicted_speed)**2)

                    
                    print('Frame {} with actual speed {} has predicted speed of {}. MSE of {}'
                          .format(frame_count,true_speed,predicted_speed,mse_error))
                    
                    write_mse='Mean-Squared Error: '+str(round((float(mse_error/(10*sum_frames+ind))),2))
                    frame=cv2.putText(frame,write_mse,(10,80), cv2.FONT_HERSHEY_SIMPLEX,0.8,(0,0,255),2)

                    vwriter.write(cv2.cvtColor(np.uint8(frame),cv2.COLOR_BGR2RGB))
                    write_prediction.write(str(predicted_speed)+'\n')
                sum_frames+=1
            count+=1
        frame_count+=1
    
    vwriter.release()
    write_prediction.close()
    print("Prediction video created.")
    break

Frame 0
Frame 1
Frame 2
Frame 3
Frame 4
Frame 5
Frame 6
Frame 7
Frame 8
Frame 9
Frame 10
Frame 11
Frame 12
Frame 13
Frame 14
Frame 15
Frame 15 with actual speed 28.105569 has predicted speed of 15.182829856872559. MSE of 166.99718696131814
Frame 15 with actual speed 28.105569 has predicted speed of 15.182829856872559. MSE of 166.99718696131814
Frame 15 with actual speed 28.106527 has predicted speed of 15.182829856872559. MSE of 167.0219478472804
Frame 15 with actual speed 28.130404 has predicted speed of 15.182829856872559. MSE of 167.63967619178226
Frame 15 with actual speed 28.109243 has predicted speed of 15.182829856872559. MSE of 167.09215674681784
Frame 15 with actual speed 28.088572 has predicted speed of 15.182829856872559. MSE of 166.55818026489567
Frame 15 with actual speed 28.034211 has predicted speed of 15.182829856872559. MSE of 165.15799728593157
Frame 15 with actual speed 28.018491 has predicted speed of 15.182829856872559. MSE of 164.75419698119168
Frame 15 with actua

/home/n_srishankar/miniconda3/envs/carnd-term1/lib/python3.5/site-packages/ipykernel_launcher.py:56: RuntimeWarning: divide by zero encountered in double_scalars


Frame 11
Frame 12
Frame 13
Frame 14
Frame 15
Frame 31 with actual speed 27.981986 has predicted speed of 17.872867584228516. MSE of 102.19427514409014
Frame 31 with actual speed 27.978625 has predicted speed of 17.872867584228516. MSE of 102.12633294642038
Frame 31 with actual speed 28.032331 has predicted speed of 17.872867584228516. MSE of 103.21469689639918
Frame 31 with actual speed 27.960219 has predicted speed of 17.872867584228516. MSE of 101.75465858526694
Frame 31 with actual speed 27.937178 has predicted speed of 17.872867584228516. MSE of 101.29034414500637
Frame 31 with actual speed 27.940599 has predicted speed of 17.872867584228516. MSE of 101.35921586011207
Frame 31 with actual speed 27.908579 has predicted speed of 17.872867584228516. MSE of 100.71550362064609
Frame 31 with actual speed 27.898606 has predicted speed of 17.872867584228516. MSE of 100.51543078147613
Frame 31 with actual speed 27.855982 has predicted speed of 17.872867584228516. MSE of 99.66257343838444
Fr

Frame 10
Frame 11
Frame 12
Frame 13
Frame 14
Frame 15
Frame 111 with actual speed 27.224821 has predicted speed of 8.870776176452637. MSE of 336.8709613847857
Frame 111 with actual speed 27.181666 has predicted speed of 8.870776176452637. MSE of 335.2886861300904
Frame 111 with actual speed 27.170096 has predicted speed of 8.870776176452637. MSE of 334.86510600447355
Frame 111 with actual speed 27.15459 has predicted speed of 8.870776176452637. MSE of 334.2978479341416
Frame 111 with actual speed 27.104113 has predicted speed of 8.870776176452637. MSE of 332.4545717209283
Frame 111 with actual speed 27.081358 has predicted speed of 8.870776176452637. MSE of 331.62529035211367
Frame 111 with actual speed 27.071874 has predicted speed of 8.870776176452637. MSE of 331.2799619823406
Frame 111 with actual speed 27.073267 has predicted speed of 8.870776176452637. MSE of 331.330672181326
Frame 111 with actual speed 27.039336 has predicted speed of 8.870776176452637. MSE of 330.0965660618194
F

Frame 6
Frame 7
Frame 8
Frame 9
Frame 10
Frame 11
Frame 12
Frame 13
Frame 14
Frame 15
Frame 191 with actual speed 26.287243 has predicted speed of 8.638635635375977. MSE of 311.4733419106613
Frame 191 with actual speed 26.286672 has predicted speed of 8.638635635375977. MSE of 311.45318752709187
Frame 191 with actual speed 26.273133 has predicted speed of 8.638635635375977. MSE of 310.9754973029317
Frame 191 with actual speed 26.258781 has predicted speed of 8.638635635375977. MSE of 310.4695226704814
Frame 191 with actual speed 26.26402 has predicted speed of 8.638635635375977. MSE of 310.65417400073295
Frame 191 with actual speed 26.216485 has predicted speed of 8.638635635375977. MSE of 308.98078828541315
Frame 191 with actual speed 26.229105 has predicted speed of 8.638635635375977. MSE of 309.42461246777634
Frame 191 with actual speed 26.205432 has predicted speed of 8.638635635375977. MSE of 308.59233451616774
Frame 191 with actual speed 26.184952 has predicted speed of 8.6386356

...

Frame 7
Frame 8
Frame 9
Frame 10
Frame 11
Frame 12
Frame 13
Frame 14
Frame 15
Frame 20031 with actual speed 9.201983 has predicted speed of 8.187545776367188. MSE of 1.0290828806918493
Frame 20031 with actual speed 9.230482 has predicted speed of 8.187545776367188. MSE of 1.0877159665654725
Frame 20031 with actual speed 9.208516 has predicted speed of 8.187545776367188. MSE of 1.0423801975448341
Frame 20031 with actual speed 9.280405 has predicted speed of 8.187545776367188. MSE of 1.1943412826793136
Frame 20031 with actual speed 9.375487 has predicted speed of 8.187545776367188. MSE of 1.411204350806223
Frame 20031 with actual speed 9.410634 has predicted speed of 8.187545776367188. MSE of 1.4959448027892686
Frame 20031 with actual speed 9.429654 has predicted speed of 8.187545776367188. MSE of 1.5428328392162594
Frame 20031 with actual speed 9.456148 has predicted speed of 8.187545776367188. MSE of 1.609351601806118
Frame 20031 with actual speed 9.594891 has predicted speed of 8.1875

Frame 5
Frame 6
Frame 7
Frame 8
Frame 9
Frame 10
Frame 11
Frame 12
Frame 13
Frame 14
Frame 15
Frame 20111 with actual speed 10.61493 has predicted speed of 7.647945880889893. MSE of 8.802994763051576
Frame 20111 with actual speed 10.646422 has predicted speed of 7.647945880889893. MSE of 8.990859036873607
Frame 20111 with actual speed 10.501995 has predicted speed of 7.647945880889893. MSE of 8.145596374293184
Frame 20111 with actual speed 10.461287 has predicted speed of 7.647945880889893. MSE of 7.9148882524757145
Frame 20111 with actual speed 10.449606 has predicted speed of 7.647945880889893. MSE of 7.849299423012058
Frame 20111 with actual speed 10.580557 has predicted speed of 7.647945880889893. MSE of 8.60020797592824
Frame 20111 with actual speed 10.472835 has predicted speed of 7.647945880889893. MSE of 7.979998535266678
Frame 20111 with actual speed 10.56318 has predicted speed of 7.647945880889893. MSE of 8.498589969223678
Frame 20111 with actual speed 10.427848 has predicte

Frame 10
Frame 11
Frame 12
Frame 13
Frame 14
Frame 15
Frame 20191 with actual speed 10.02409 has predicted speed of 6.254762172698975. MSE of 14.207832269665863
Frame 20191 with actual speed 10.012719 has predicted speed of 6.254762172698975. MSE of 14.122239515858393
Frame 20191 with actual speed 10.066996 has predicted speed of 6.254762172698975. MSE of 14.533126754018221
Frame 20191 with actual speed 10.043765 has predicted speed of 6.254762172698975. MSE of 14.356542425295167
Frame 20191 with actual speed 10.083832 has predicted speed of 6.254762172698975. MSE of 14.661775742347098
Frame 20191 with actual speed 10.075386 has predicted speed of 6.254762172698975. MSE of 14.597166429740335
Frame 20191 with actual speed 10.062279 has predicted speed of 6.254762172698975. MSE of 14.497184390180468
Frame 20191 with actual speed 10.149821 has predicted speed of 6.254762172698975. MSE of 15.171483268135633
Frame 20191 with actual speed 10.126287 has predicted speed of 6.254762172698975. M

Frame 10
Frame 11
Frame 12
Frame 13
Frame 14
Frame 15
Frame 20271 with actual speed 10.592486 has predicted speed of 5.3055620193481445. MSE of 27.951565177191654
Frame 20271 with actual speed 10.600408 has predicted speed of 5.3055620193481445. MSE of 28.035393958825107
Frame 20271 with actual speed 10.599798 has predicted speed of 5.3055620193481445. MSE of 28.02893461882871
Frame 20271 with actual speed 10.611157 has predicted speed of 5.3055620193481445. MSE of 28.149338098718168
Frame 20271 with actual speed 10.622889 has predicted speed of 5.3055620193481445. MSE of 28.273966219168184
Frame 20271 with actual speed 10.603359 has predicted speed of 5.3055620193481445. MSE of 28.06665284820391
Frame 20271 with actual speed 10.608069 has predicted speed of 5.3055620193481445. MSE of 28.11658027986166
Frame 20271 with actual speed 10.604959 has predicted speed of 5.3055620193481445. MSE of 28.083608358541994
Frame 20271 with actual speed 10.614746 has predicted speed of 5.305562019348

Frame 9
Frame 10
Frame 11
Frame 12
Frame 13
Frame 14
Frame 15
Frame 20351 with actual speed 10.564073 has predicted speed of 6.595147132873535. MSE of 15.752372538745565
Frame 20351 with actual speed 10.53326 has predicted speed of 6.595147132873535. MSE of 15.508732954227028
Frame 20351 with actual speed 10.532227 has predicted speed of 6.595147132873535. MSE of 15.500597880132547
Frame 20351 with actual speed 10.535906 has predicted speed of 6.595147132873535. MSE of 15.529580448835864
Frame 20351 with actual speed 10.536697 has predicted speed of 6.595147132873535. MSE of 15.535815355044654
Frame 20351 with actual speed 10.525051 has predicted speed of 6.595147132873535. MSE of 15.444144404855539
Frame 20351 with actual speed 10.52872 has predicted speed of 6.595147132873535. MSE of 15.472995500993516
Frame 20351 with actual speed 10.533034 has predicted speed of 6.595147132873535. MSE of 15.506952978287089
Frame 20351 with actual speed 10.536988 has predicted speed of 6.59514713287

In [ ]:
print("Script terminated at",str(datetime.now()))